# Oleveler example for transcriptomics data analysis

### General module import

In [ ]:
%load_ext autoreload
%autoreload 2
try:
    from oleveler import *
except:
    import os
    os.symlink('../oleveler.py', 'oleveler.py')
    from oleveler import *

# # For interactive plots (PCA, PLS, Loading, volcano), the plots will not be saved if you export this notebook.
# %matplotlib widget

# # For exproting this notebook with plots to HTML
%matplotlib inline

# # You can change plotting style to dark or white using these functions
# setDarkModePlotting(forceDark=True)
# setDarkModePlotting(forceWhite=True)

# # Logging level: INFO = more text shown; WARNING = less text. All INFO level message are anyway stored in the dataProcessing.log file.
logshandler.setLevel(logging.INFO)
# logshandler.setLevel(logging.WARNING)

## Data processing

Note the file/folder path is the parameters of the function, please change accroding to your file/folder name.

In [ ]:
# Load data
selfAlignCt = gatherCountTable("featureCounts/")
saDf = calculateTPM(selfAlignCt, 'featureCounts/GCF_000203835.1_ASM20383v1_genomic.gff', 
                    tagsForGeneName='locus_tag', removerRNA=True, removeIDcontains=['SCP'])

# Load meta
metaDf, conditions, experiments = loadMeta('Annotation.csv')

# Calculate mean and var
meanDf, nquantDf, varDf, stdDf, semDf = getStats(saDf, experiments)

# Transformation vst using DESeq2
# https://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#data-transformations-and-visualization
vstDf = deseq2Process(selfAlignCt, metaDf, ref='WT_45')
# Calculate mean and var
# These data should only be used in plotting, principal analysis, or other stastistical analyses.
vstMeanDf, vstNquantDf, vstVarDf, vstStdDf, vstSemDf = getStats(vstDf, experiments, title='vst')

### DESeq2 different expression analysis

In [ ]:
# Differential expression analysis
# https://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#differential-expression-analysis
deseq2CompResults, comparisons = makeCompMatrixDeseq2('comparisons.xlsx', 
                                                      selfAlignCt,
                                                      'annotation.csv',
                                                      shrink=None)

In [ ]:
# Differential expression analysis with log fold change shrinkage for visualization and ranking
# https://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#log-fold-change-shrinkage-for-visualization-and-ranking
deseq2CompResultsShrink, comparisons = makeCompMatrixDeseq2('comparisons.xlsx', 
                                                            selfAlignCt,
                                                            'annotation.csv',
                                                            shrink='lfcShrink')

## General analysis

### Principal components analysis (PCA)

Unsupervised data reduction.

In [ ]:
# I have a gene knockout in the mutant, so it is slightly better if I remove data of this gene before analysis
newVstDf = vstDf[vstDf.index != 'SCO1839']

# Do a PCA plot
PcaClass_vst = plotPrincipleAnalysis(newVstDf, colourSet=genColour(vstDf.columns), figsize=(7,6),
                                     title='VST df (no gbn)', analysisType='PCA')
# Get some info about the PCA result just generated
plotPCAExplanation(PcaClass_vst)

# Loading plot of the same PCA. Note the loading plot function do not dependent on the PCA result before.
plotPrincipleAnalysisLoading(newVstDf, drawOutliers=True, outlierAlg=0, outliersFraction=0.05, title='VST df (no gbn)')

You can also customise data for any analysis on the fly. eg. Use only part of the data by constraining the columns of the data table.

In [ ]:
cols = [
    'C24_1', 'C24_2', 'C24_3', 
#     'C45_1', 'C45_2', 'C45_3', 
    'D24_1', 'D24_2', 'D24_3', 
#     'D45_1', 'D45_2', 'D45_3',
]
plotPrincipleAnalysis(newVstDf, cols=cols, title='VST 24h only', colourSet=genColour(cols))

### Partial least squares regression (PLS regression)

Supervised data reduction.

In [ ]:
# First make a class table based on your data. Describe your conditions in numbers.
# In this example, the first column (0 and 2) indicates wild type and mutant (strains),
# the second column (24 and 45) indicates time after inoculation. You can use numbers with
# larger difference to give specific element higher impact.
plsClasses = [
              [0,24],
              [0,24],
              [0,24],
              [0,45],
              [0,45],
              [0,45],
              [2,24],
              [2,24],
              [2,24],
              [2,45],
              [2,45],
              [2,45],
]

# Do PLS and plot. analysisType='PLS'
PlsClass_vst = plotPrincipleAnalysis(vstDf,
                                     colourSet=genColour(vstDf.columns),
                                     title='VST df', analysisType='PLS', plsClasses=plsClasses)

# V plot of PLS showing the importance of each genes contributing to the seperation.
plotPlsVplot(newVstDf, classes=plsClasses, title='VST', outlierAlg=1)

PLS for part of the data is also doable.

In [ ]:
# Simply comment out the non-required data from both classes info and columns of data frame.
plsClasses = [
              [0,24],
              [0,24],
              [0,24],
#               [0,45],
#               [0,45],
#               [0,45],
              [2,24],
              [2,24],
              [2,24],
#               [2,45],
#               [2,45],
#               [2,45],
]
cols = [
    'C24_1', 'C24_2', 'C24_3', 
#     'C45_1', 'C45_2', 'C45_3', 
    'D24_1', 'D24_2', 'D24_3', 
#     'D45_1', 'D45_2', 'D45_3',
]
PlsClass_vst = plotPrincipleAnalysis(newVstDf,cols=cols, colourSet=genColour(vstDf.columns),
                                     title='VST 24h only', analysisType='PLS', plsClasses=plsClasses)

### Correlation plot

In [ ]:
plotCorr(vstDf, cols=[c for c in vstDf if c!='QC'], vmin=0.8)

### Volcano plot for each comparison

In [ ]:
# First let us see how many comparison pairs are stored in the comparison results
comparisons

It is possible to highlight selected genes in the volcano plot.

In [ ]:
# for c in comparisons:
for c in ['mu_wt24']:
    cols = metaDf[(metaDf.Condition==comparisons[c]['exp']) | \
                  (metaDf.Condition==comparisons[c]['ctr'])].Experiment.values
    plotVolcano(deseq2CompResultsShrink[c],
                # Second argument is data used for the size of each point on the plot. This is optional.
                # If not provided, it will try to use the "mean" column from comparison result.
                vstDf[cols].mean(axis=1), 
                xmax = 6, ymax=80,
                # Highlight is achieved by passing an OrderedDict (or Dict), list of lists to this highlights parameter
                highlights=OrderedDict([
                    ('GlnR regulon' , ['SCO2198', # glnA
                            'SCO5583', 'SCO5584', 'SCO5585', # amtB, glnK, glnD
                            'SCO2210', # glnII
                            'SCO4683', # gdhA
                            'SCO2486','SCO2487', 'SCO2488', # nirB_a, nirB_b, nirCD
                            'SCO1236', 'SCO1235', 'SCO1234', 'SCO1233',# ureA, B, C, F
                            'SCO0255', 'SCO0888', 'SCO2400', 'SCO2404', 'SCO1863', 'SCO2195', 'SCO7155'
                           ]),
                    (r'$act$',[f'SCO{i}' for i in range(5071, 5093)]), # Latex syntex is supported in the name
                    (r'$nuo$JKLMN', [f'SCO{i}' for i in range(4571, 4576)]),
                    (r'$nuo2$'  ,[f'SCO{i}' for i in range(4599, 4609)]),
                    (r"$bxl$AEFG", [f'SCO{i}' for i in range(7028,7032)]),
                    ("carboxylesterase related", [f'SCO0{i}' for i in range(319,322)]),
                    ("SoxR regulon", ['SCO4265', 'SCO4266','SCO2477','SCO2478', 'SCO7008', 'SCO1909', 'SCO1178']),
                    ('Possible HGT cluster',[f'SCO{i}' for i in range(4615, 4628)]),
                ]),
                figsize=(7,7),
                title=f'shrink_comp_result_{c}')

In [ ]:
for c in ['mu_wt45']:
    cols = metaDf[(metaDf.Condition==comparisons[c]['exp']) | \
                  (metaDf.Condition==comparisons[c]['ctr'])].Experiment.values
    plotVolcano(deseq2CompResultsShrink[c], 
                vstDf[cols].mean(axis=1), 
                highlights=OrderedDict([
                    ('GlnR regulon' , ['SCO2198', # glnA
                            'SCO5583', 'SCO5584', 'SCO5585', # amtB, glnK, glnD
                            'SCO2210', # glnII
                            'SCO4683', # gdhA
                            'SCO2486','SCO2487', 'SCO2488', # nirB_a, nirB_b, nirCD
                            'SCO1236', 'SCO1235', 'SCO1234', 'SCO1233',# ureA, B, C, F
                            'SCO0255', 'SCO0888', 'SCO2400', 'SCO2404', 'SCO1863', 'SCO2195', 'SCO7155'
                           ]),
                    (r'$act$',[f'SCO{i}' for i in range(5071, 5093)]),
                    (r'$nuo$JKLMN', [f'SCO{i}' for i in range(4571, 4576)]),
                    (r'$nuo2$'  ,[f'SCO{i}' for i in range(4599, 4609)]),
                    (r"$bxl$AEFG", [f'SCO{i}' for i in range(7028,7032)]),
                    ("carboxylesterase related", [f'SCO0{i}' for i in range(319,322)]),
                    ("SoxR regulon", ['SCO4265', 'SCO4266','SCO2477','SCO2478', 'SCO7008', 'SCO1909', 'SCO1178']),
                    ('Possible HGT cluster',[f'SCO{i}' for i in range(4615, 4628)]),
#                     ('Nit' ,[f'SCO{i}' for i in range(5583, 5586)]),
#                     ('Nir' ,[f'SCO{i}' for i in range(2486, 2489)]),
#                     ('pGlnr' , ['SCO2471', 'SCO2472', 'SCO2473']),
#                     ('Act',[f'SCO{i}' for i in range(5071, 5093)]),
#                     ('red',[f'SCO{i}' for i in range(5877, 5899)]),
#                     ('cpk',[f'SCO{i}' for i in range(6273, 6289)]),
#                     ('whi',[f'SCO{i}' for i in [3034,4543,4767,5621,6029,1950,5314,5315,5316,5317,5318,5319,5320,5321,5819]]),
#                     ('bld',[f'SCO{i}' for i in [1489,3323,3549,4091,4768,5112,5113,5114,5115,5116,5723,2792]])
                ]),
                figsize=(7,7),
                xmax = 6, ymax=160,
                title=f'shrink_comp_result_{c}')

### Expression profile clustering (co-expression analysis)

Heat map and expression profile of each cluster.

In [ ]:
# First get the desired data set: for this project,
# I select to cluster the genes that significantly changed
# in the comparison between mutant and wild type at 45 hours.
sigDf = getSig(deseq2CompResultsShrink['mu_wt45'], tPv=0.01)

# use all data
cols = ['C24_1', 'C24_2', 'C24_3', 'C45_1', 'C45_2', 'C45_3',
        'D24_1', 'D24_2', 'D24_3', 'D45_1', 'D45_2', 'D45_3']

cluster, fname = plotHeatmapGetCluster(vstDf, sigDf.index, cols,
                                       nCluster=20,
                                       saveFig=True,
                                       ylabels="AUTO",
                                       standard_scale='row',
                                       plot=True, title='sig_mu_wt45')

In [ ]:
plotCluster(cluster, fname,
            saveFig=True,
            conditions = [['C24_1', 'C24_2', 'C24_3', 'C45_1', 'C45_2', 'C45_3'],
                          ['D24_1', 'D24_2', 'D24_3', 'D45_1', 'D45_2', 'D45_3']],
            dataLabels = ['WT', 'Mut'],
            xlabels = ['24_1', '24_2', '24_3', '45_1', '45_2', '45_3'],
            longWide='long',
            xlabelRotation=45)

You can also choose to plot only interesting clusters:

In [ ]:
plotCluster(cluster, fname,
            saveFig=True,
            clusters=[1,14,11,18,19],
            conditions = [['C24_1', 'C24_2', 'C24_3', 'C45_1', 'C45_2', 'C45_3'],
                          ['D24_1', 'D24_2', 'D24_3', 'D45_1', 'D45_2', 'D45_3']],
            dataLabels = ['WT', 'Mut'],
            xlabels = ['24_1', '24_2', '24_3', '45_1', '45_2', '45_3'],
            longWide='long',
            xlabelRotation=45)

## Query genes

### Many genes and many conditions

In [ ]:
queryGenes = ['SCO1839', 'SCO4117']

queryConditions = ['Dgbn_24', 'Dgbn_45', 'WT_24', 'WT_45'  ]
query(meanDf, semDf, queryGenes, queryConditions, 
      xlabels=[r'${\Delta}gbn$ 24h', r'${\Delta}gbn$ 45h', 
               'WT 24h',            'WT 45h'  ],
      xlabelRotation=30, plotType='bar')
      # xlabelRotation=30, plotType='line fill')
      # xlabelRotation=30, plotType='line bar')

query(meanDf, semDf, queryGenes, queryConditions, 
      xlabels=[r'${\Delta}gbn$ 24h', r'${\Delta}gbn$ 45h', 
               'WT 24h',            'WT 45h'  ],
      # xlabelRotation=30, plotType='bar')
      xlabelRotation=30, plotType='line fill')
      # xlabelRotation=30, plotType='line bar')

query(meanDf, semDf, queryGenes, queryConditions, 
      xlabels=[r'${\Delta}gbn$ 24h', r'${\Delta}gbn$ 45h', 
               'WT 24h',            'WT 45h'  ],
      # xlabelRotation=30, plotType='bar')
      # xlabelRotation=30, plotType='line fill')
      xlabelRotation=30, plotType='line bar')


If you have a series of genes to query

In [ ]:
soxRRegulon = ['SCO4265', 'SCO4266','SCO2477','SCO2478', 'SCO7008', 'SCO1909', 'SCO1178']
queryConditions = ['Dgbn_24', 'Dgbn_45',   'WT_24', 'WT_45'  ]
query(meanDf, semDf, soxRRegulon, queryConditions, 
      xlabels=[r'${\Delta}gbn$ 24h', r'${\Delta}gbn$ 45h', 
               'WT 24h',            'WT 45h'  ],
      xlabelRotation=30, plotType='bar')
#       xlabelRotation=30, plotType='line fill')
#       xlabelRotation=30, plotType='line bar')


### Single gene, reduce one factor (eg. combine strains)

In [ ]:
# Crystal structure of the solute-binding protein 
# BxlE from Streptomyces thermoviolaceus OPC-520 complexed with xylobiose
queryGenes = 'SCO7028'

# Put the conditions you want to put together in the same column
queryConditions = [['Dgbn_24','Dgbn_45'],
                   ['WT_24',  'WT_45'  ]]
query(meanDf, semDf, queryGenes,
      queryConditions,
      queryConditionGroupNames=[[r'M145${\Delta}gbn$','M145'],[24,45]],
      xlabelRotation=30,
      plotType='bar')
      # plotType='line fill')
      # plotType='line bar')

query(meanDf, semDf, queryGenes,
      queryConditions,
      queryConditionGroupNames=[[r'M145${\Delta}gbn$','M145'],[24,45]],
      xlabelRotation=30,
      plotType='line fill')
      # plotType='line bar')

query(meanDf, semDf, queryGenes,
      queryConditions,
      queryConditionGroupNames=[[r'M145${\Delta}gbn$','M145'],[24,45]],
      xlabelRotation=30,
      # plotType='line fill')
      plotType='line bar')

It is also possible to use only part of the conditions. eg. the gene SCO1839 has been knocked out in the mutant, so it is not necessary to show its expression.

In [ ]:
queryConditions = ['WT_24', 'WT_45'  ]
query(meanDf, semDf, 'SCO1839', queryConditions, 
      xlabels=['WT 24h', 'WT 45h'],
      figsize=(4, 4),
      plotType='bar', square=True,
      width=0.2,
     )
#       xlabelRotation=30, plotType='line fill')
#       xlabelRotation=30, plotType='line bar')


---

## Export R session infomation (package versions etc.)

In [ ]:
writeRSessionInfo('RSessionInfo.txt', overwrite=False)